In [1]:
import ipynb.fs.full.feed_reader as fr
import ipynb.fs.full.data_operations as do
import ipynb.fs.full.dict_maker as dictionary
import ipynb.fs.full.backtester as bt
import pandas
import datetime
import json

In [2]:
def get_article_links(sub_site: str):
    url = dictionary.dict_to_json.get_value_from_file(sub_site)        #Get the url for this sub-site
    feeder = fr.livemint_reader.read_feed(url)                          #Get all the links for feed for this sub-site as a DataFrame
    feeder.insert(1, 'sub_site_name', 'Markets')
    print(f'Found {feeder.shape[0]} links. Now deduping...')
    db = do.mysql_Database('analyst_feeds')                                            #Initialize the MySQL DB
    links_seen = db.search_values_in_table('feeds_seen', 'link_id', feeder['link_id'], ['site_name', 'sub_site_name'], ['Livemint', 'Markets'], ['link_id'])       #Get values which are already in the table
    print(f'Found {links_seen.shape[0]} links which have been analysed before...')
    links_to_add = feeder[~feeder['link_id'].isin(links_seen['link_id'])]                    #remove values already in the table
    print(f'Inserting {links_to_add.shape[0]} new links to the table...')
    db.insert_values_in_table('feeds_seen', links_to_add[['link_id', 'site_name', 'sub_site_name', 'link', 'link_date']])                                         #Insert table values of links seen so as to not repeat
    db.extinguish_connection()
    print('DB connection closed. Exiting...')

In [3]:
def insert_prev_recos_in_table():
    recos = pandas.read_csv('../../data/prev_recos.csv', sep=',', na_values=['None'])
    recos['reco_date'] = pandas.to_datetime(recos['reco_date']).dt.strftime('%Y-%m-%d')
    recos.replace('None', None, inplace=True)
    #print(recos.head)
    db = do.mysql_Database('analyst_feeds')
    db.insert_values_in_table('reco_repository', recos)

In [4]:
def run_backtests_on_all_recos(analyst:str):
    db = do.mysql_Database('analyst_feeds')
    recos = db.search_values_in_table(tablename='reco_repository', test_field='analyst', match_values=pandas.Series(analyst), aux_fields=['site_name'], aux_field_values=['Livemint'])
    
    for idx, row in recos.iterrows():
        try:
            tester = bt.ticker_Data(ticker=row['ticker'], start_date=row['reco_date'].strftime('%Y-%m-%d'), in_price=row['reco_value'], tgt_price=row['tgt_value'], sl_price=row['sl_value'])
            if(tester.sanity_test(row)):
                price_data = tester.get_price_data()
                tester.simple_win_or_loss(price_data)
                insert_df = pandas.DataFrame(
                        data={
                            'reco_id': [row['reco_id']],
                            'ticker': [row['ticker']],
                            'testable': [1],
                            'outcome': [json.dumps(tester.outcome)],
                            'result': [tester.outcome.get('result')],
                            'result_price': [tester.outcome_price],
                            'result_horizon': [tester.return_horizon],
                            'result_return': [tester.return_rate]
                        }
                    )
                db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
            else:
                #print(f'Sanity test for row {idx} for ticker {tester.ticker} at start_date {tester.start_date} is False')
                insert_df = pandas.DataFrame(
                        data={
                            'reco_id': [row['reco_id']],
                            'ticker': [row['ticker']],
                            'testable': [0]
                        }
                    )
                db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
        except Exception as e:
            print(f'Caught exception {e} in processing row {idx} with reco_id {row['reco_id']}.')
            insert_df = pandas.DataFrame(
                        data={
                            'reco_id': [row['reco_id']],
                            'ticker': [row['ticker']],
                            'testable': [1],
                            'result': ['failure'],
                        }
                    )
            db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
        finally:
            continue

In [5]:
if __name__ == '__main__':
    #hl = get_article_links('AI')
    #insert_prev_recos_in_table()
    run_backtests_on_all_recos('Sumeet Bagadia')

Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.
Inserted 1 values in the reco_backtest_results.


[*********************100%***********************]  1 of 1 completed


Inserted 1 values in the reco_backtest_results.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Inserted 1 values in the reco_backtest_results.


...

Inserted 1 values in the reco_backtest_results.



[*********************100%***********************]  1 of 1 completed

Writing DataFrame to database caused an error. (mysql.connector.errors.DataError) 1406 (22001): Data too long for column 'result' at row 1
[SQL: INSERT INTO reco_backtest_results (reco_id, ticker, testable, outcome, result, result_price, result_horizon, result_return) VALUES (%(reco_id)s, %(ticker)s, %(testable)s, %(outcome)s, %(result)s, %(result_price)s, %(result_horizon)s, %(result_return)s)]
[parameters: {'reco_id': '38cada2f-359d-11f0-9ead-0a4228b31e6b', 'ticker': 'SILGO', 'testable': 1, 'outcome': '{"result": "undet", "index": "2025-05-20 09:45:00 UTC+0000"}', 'result': 'undet', 'result_price': 49.31, 'result_horizon': 60, 'result_return': 0.0}]
(Background on this error at: https://sqlalche.me/e/20/9h9h)



[*********************100%***********************]  1 of 1 completed

Writing DataFrame to database caused an error. (mysql.connector.errors.DataError) 1406 (22001): Data too long for column 'result' at row 1
[SQL: INSERT INTO reco_backtest_results (reco_id, ticker, testable, outcome, result, result_price, result_horizon, result_return) VALUES (%(reco_id)s, %(ticker)s, %(testable)s, %(outcome)s, %(result)s, %(result_price)s, %(result_horizon)s, %(result_return)s)]
[parameters: {'reco_id': '38cada74-359d-11f0-9ead-0a4228b31e6b', 'ticker': 'ATALREAL', 'testable': 1, 'outcome': '{"result": "undet", "index": "2025-05-20 09:45:00 UTC+0000"}', 'result': 'undet', 'result_price': 15.34, 'result_horizon': 60, 'result_return': 0.0}]
(Background on this error at: https://sqlalche.me/e/20/9h9h)
Writing DataFrame to database caused an error. (mysql.connector.errors.DataError) 1406 (22001): Data too long for column 'result' at row 1
[SQL: INSERT INTO reco_backtest_results (reco_id, ticker, testable, outcome, result, result_price, result_horizon, result_return) VALUES 

In [21]:
import json
db = do.mysql_Database('analyst_feeds')
df = pandas.DataFrame(
            data = {
                'reco_id': ['aaa-bb-ccc-ddd-eee-fff'],
                'testable': [1],
                'result': ['win'],
                'outcome': [str(json.dumps('{\'result\': \'lose\', \'index\': Timestamp(\'2023-07-07 06:45:00+0000\', tz=\'UTC\')}'))]
            }
        )
db.insert_values_in_table(tablename='reco_backtest_results', df=df)
#print(str(json.dumps('{\'result\': \'lose\', \'index\': Timestamp(\'2023-07-07 06:45:00+0000\', tz=\'UTC\')}')))

Inserted 1 values in the reco_backtest_results.
